Использовать датасет MovieLens.
Построить рекомендации (регрессия, предсказываем оценку) на фичах:
TF-IDF на тегах и жанрах;
средние оценки (+ median, variance и т. д.) пользователя и фильма.
Оценить RMSE на тестовой выборке.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
/content/drive/MyDrive/rek_syst/ml-latest-small

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer;
#links = pd.read_csv('ml-latest/links.csv')
movies = pd.read_csv('/content/drive/MyDrive/rek_syst/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/rek_syst/ml-latest-small/ratings.csv')
tags = pd.read_csv('/content/drive/MyDrive/rek_syst/ml-latest-small/tags.csv')
def change_string(s):
    return str(s).replace(' ', '').replace('-', '').replace("\n",'|').lower()
tags = tags.drop('userId',axis=1).drop('timestamp',axis=1)

### средняя оценка

In [ ]:
mean_userrating = ratings.groupby(['userId']).mean('rating').drop('movieId',axis=1).drop('timestamp',axis=1) #средняя оценка пользователя
mean_movierating = ratings.groupby(['movieId']).mean('rating').drop('userId',axis=1).drop('timestamp',axis=1) #средняя оценка фильма

In [ ]:
mean_userrating.head()

,rating
userId,
1,4.366379
2,3.948276
3,2.435897
4,3.555556
5,3.636364


In [ ]:
mean_movierating.head()

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


### объединил теги и фильмы

In [ ]:

tags= tags.groupby('movieId')['tag'].apply(list).reset_index()
tags['tag'] = tags['tag'].astype(str)
movies_and_tag = tags.merge(movies, how = 'right', on= 'movieId')
movies_and_tag = movies_and_tag[['movieId','title','genres','tag']]

### tf-idf жанры

In [ ]:
movie_genres = movies_and_tag.genres.apply(change_string).tolist()
tfidf = TfidfVectorizer()
genresX_train_tfidf = tfidf.fit_transform(movie_genres)
tfidfgenres = pd.DataFrame(genresX_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())

### tf-idf тэги

In [ ]:
movie_tag = movies_and_tag.tag.apply(change_string).tolist()
tfidf_tag = TfidfVectorizer()
tagX_train_tfidf = tfidf_tag.fit_transform(movie_tag)
tfidftag = pd.DataFrame(tagX_train_tfidf.toarray(), columns=tfidf_tag.get_feature_names_out())

### Объединяем tf-idf

In [ ]:
columns1=tfidf.get_feature_names_out()
columns2=tfidf_tag.get_feature_names_out()
columns=np.hstack([columns1,columns2])
body = np.hstack([genresX_train_tfidf.toarray(),tagX_train_tfidf.toarray()])
unitedtfidf = pd.DataFrame(body, columns=columns)

In [ ]:
mid = movies_and_tag.drop('title',axis=1).drop('genres',axis=1).drop('tag',axis=1)
unitedtfidf = mid.join(unitedtfidf)
unitedtfidf = unitedtfidf.merge(mean_movierating['rating'],how = 'left', on = 'movieId')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [ ]:
unitedtfidf = unitedtfidf.fillna(0)

In [ ]:
X = unitedtfidf
y = unitedtfidf['rating']
del X ['movieId']
del X ['rating']
#X = X.to_numpy()
#y = y.to_numpy()


In [ ]:
train_X, test_X, train_y , test_y = train_test_split(X, y, test_size = 0.2, random_state = 13)

In [ ]:
model = Ridge(max_iter=1000000, alpha = 13)

In [ ]:
model.fit(train_X,train_y)

Ridge(alpha=13, max_iter=1000000)

In [ ]:
predict = model.predict(test_X)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(predict, test_y)

0.6338970254827551

In [ ]:
predict

array([3.5601973 , 3.44511323, 3.35707297, ..., 3.38769247, 3.05821246,
       3.09347481])

In [ ]:
test_y.to_numpy()

array([3.5       , 4.        , 3.1       , ..., 3.8125    , 2.71428571,
       0.5       ])